In [1]:
import pandas as pd
from tqdm import tqdm
import json
import os

In [2]:
# load /hpc2hdd/home/yxu409/MicroLens/Dataset/Microlens-100k/MicroLens-100k_comment_en.txt
comments = pd.read_csv('/hpc2hdd/home/yxu409/MicroLens/Dataset/Microlens-100k/MicroLens-100k_comment_en.txt', sep='	', header=None)

/tmp/ipykernel_786142/3659970425.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv('/hpc2hdd/home/yxu409/MicroLens/Dataset/Microlens-100k/MicroLens-100k_comment_en.txt', sep='	', header=None)


In [3]:
comments.columns = ['user_id', 'video_id', 'comment']

In [4]:
timechat_results_dir = '/hpc2hdd/home/yxu409/MicroLens/TimeChat/results'

In [5]:
all_timechat_files = []
for filename in os.listdir(timechat_results_dir):
    with open(os.path.join(timechat_results_dir, filename), 'r') as file:
        for line in file:
            all_timechat_files.append(json.loads(line))

In [6]:
all_timechat_files = pd.DataFrame(all_timechat_files)

In [7]:
all_timechat_files

,video_id,message
0,9865,"0.0 - 12.8 seconds, Cartoon characters are dan..."
1,9866,This emphasizes the importance of using an app...
2,9867,"0.0 - 16.9 seconds, There are a lot of men out..."
3,9868,"0.0 - 5.0 seconds, There's an intro to a video..."
4,9869,"0.0 - 67.5 seconds, A sun rises in a beach. 1..."
...,...,...
16719,8215,"0.0 - 5.0 seconds, A woman is seen in front of..."
16720,8216,"In the given video, the highlights and feature..."
16721,8217,"0.0 - 21.2 seconds, A cartoon bear stands on t..."
16722,8218,This is a cartoon video. \n\nThe highlights ar...


In [25]:
# For range(1,19738) not in video_id, print
lacks = []
for i in range(1, 19738):
    if i not in all_timechat_files['video_id'].unique():
        lacks.append(i)

In [27]:
len(lacks)

3520

In [15]:
def searchTimechatMessages(video_id):
    if video_id in all_timechat_files.index:
        results = all_timechat_files.loc[video_id, 'message']
        if isinstance(results, str):
            return results
        else:
            return results[results.str.len().idxmax()]
    else:
        return None


In [20]:
for row, comment in tqdm(comments.iterrows(), total=comments.shape[0]):
    comments.at[row, 'content'] = searchTimechatMessages(comment['video_id'])

AttributeError: 'Series' object has no attribute 'progress_apply'

In [22]:
comments

,user_id,video_id,comment,content
0,36121,9580,@ShiChenChen This sister is really delivering ...,None
1,26572,9580,I've had a couple of times when I've accidenta...,None
2,94805,9580,"Well, why are they looking to the right I enco...",None
3,37550,9580,"No, what I'm missing is a good cell phone. [ev...",None
4,89825,9580,"Even if it's a god cell phone card, it's still...",None
...,...,...,...,...
556137,27203,7291,19 hours ago @ the,"In this video, I see highlights such as a man ..."
556138,29261,19649,It's the same thing I told you.,None
556139,28341,19188,"Don't look, this is a secondary girlfriend and...",None
556140,38967,7254,@HalfSpoonBubbleGum @Anonyme,"0.0 - 3.5 seconds, A dog is seen standing in a..."


In [21]:
comments.to_csv('/hpc2hdd/home/yxu409/MicroLens/Dataset/Microlens-100k/MicroLens-100k_comment_en_with_timechat.txt', sep='	', header=False, index=False)